In [56]:
model_name = 'shakespeare-10-30.pl'
word2id_name = 'shakespeare-10-30-word2id.pl'
models_dir = 'models/'

In [47]:
from src.models import Skipgram
import torch
import torch.nn.functional as F

In [59]:
word2id = torch.load(models_dir + word2id_name)
id2word = {i: word for word, i in word2id.items()}
model = Skipgram(len(word2id), 10)
model.load_state_dict(torch.load(models_dir + model_name))

/tmp/ipykernel_8419/3702577854.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  word2id = torch.load(models_dir + word2id_name)
/tmp/ipykernel_8419/3702577854.py:4: Futur

<All keys matched successfully>

In [63]:
word2id

{'schoolboys': 0,
 'shalt': 1,
 'crooked': 2,
 'st': 3,
 'shield': 4,
 'roman': 5,
 'princely': 6,
 'welcome': 7,
 'cicatrices': 8,
 'ministers': 9,
 'contagion': 10,
 'rank': 11,
 'fires': 12,
 'chiefest': 13,
 'oaks': 14,
 'world': 15,
 'saw': 16,
 'repent': 17,
 'threatening': 18,
 'red': 19,
 'mouth': 20,
 'needless': 21,
 'attended': 22,
 'drove': 23,
 'scandal': 24,
 'wont': 25,
 'aches': 26,
 'traitor': 27,
 'doors': 28,
 'famish': 29,
 'rushes': 30,
 'met': 31,
 'slain': 32,
 'conspectuities': 33,
 'enters': 34,
 'reeking': 35,
 'cool': 36,
 'favour': 37,
 'intent': 38,
 'bring': 39,
 'recompense': 40,
 'sun': 41,
 'beheld': 42,
 'hares': 43,
 'hold': 44,
 'press': 45,
 'innovator': 46,
 'waters': 47,
 'left': 48,
 'eleven': 49,
 'support': 50,
 'friendly': 51,
 'virgin': 52,
 'say': 53,
 'roof': 54,
 'gentlemen': 55,
 'fester': 56,
 'humbler': 57,
 'cares': 58,
 'digest': 59,
 'aufidius': 60,
 'commit': 61,
 'rivers': 62,
 'placeth': 63,
 'slip': 64,
 'naught': 65,
 'revengefu

In [66]:
find_closest('mammocked')

[(0.9537764191627502, 'tear'),
 (0.9193578958511353, 'teeth'),
 (0.9070670008659363, 'mummers'),
 (0.8878757953643799, 'blows'),
 (0.8796648979187012, 'flag'),
 (0.8755666613578796, 'flat'),
 (0.8748644590377808, 'carries'),
 (0.8710092902183533, 'sinking'),
 (0.8680746555328369, 'afternoon'),
 (0.8667041659355164, 'warrant'),
 (0.8592164516448975, 'burdens'),
 (0.8559402823448181, 'clean'),
 (0.8476640582084656, 'trim'),
 (0.8468748331069946, 'ostentation'),
 (0.8446014523506165, 'husewife'),
 (0.832159698009491, 'guess'),
 (0.82390296459198, 'pot'),
 (0.8235979676246643, 'cleave'),
 (0.8233397006988525, 'stayed'),
 (0.8205543756484985, 'trusts')]

In [61]:
def find_closest(target, model=model, word2id=word2id, id2word=id2word, top_k=20):
    target_idx = word2id[target]
    target_embed = model.embed(torch.tensor([target_idx]))  # Shape (1, embedding_dim)
    
    # Get all embeddings at once
    all_embeds = model.embed.weight  # Shape (num_embeddings, embedding_dim)
    
    # Compute cosine similarities between target and all other embeddings
    similarities = F.cosine_similarity(target_embed, all_embeds, dim=1)  # Shape (num_embeddings,)
    
    # Set similarity of the target word to a very low value to exclude it from results
    similarities[target_idx] = -float('inf')
    
    # Get the top K most similar embeddings
    top_k_indices = torch.topk(similarities, top_k).indices  # Shape (top_k,)
    
    # Retrieve corresponding words and similarities
    closest_words = [(similarities[i].item(), id2word[int(i)]) for i in top_k_indices]
    
    return closest_words